This module is responsible for:

*   Classifying semantically topic-related documents using different classifiers

**Note:**

Set the dataset path using **mr8Similar** variable 





In [ ]:

topicNumber = 8

# Configurations

In [ ]:
import numpy as np
import pandas as pd 

import string
from __future__ import print_function

import keras
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, Dropout, Embedding, LSTM, GRU, Bidirectional, Activation, Conv1D, MaxPooling1D, GlobalMaxPooling1D, BatchNormalization
from keras.optimizers import SGD
from keras.layers import SpatialDropout1D, Flatten
from keras.callbacks import Callback
from keras.callbacks import ReduceLROnPlateau
from keras import backend
from keras.initializers import Constant
from keras.regularizers import l2
from sklearn.metrics import precision_recall_fscore_support as score

from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

from matplotlib import pyplot as plt
from matplotlib.pyplot import plot



import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from IPython.display import clear_output 


In [ ]:
w2vMovieTxt = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/w2vMovie-MR-99-07-25.txt'
w2vMovieM = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/w2vMovie-MR-99-07-25.h5'

lstmModelTxt = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/lstm-99-07-25.txt'
lstmModelH = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/lstm-99-07-25.h5'

gruModelTxt = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/gru-99-07-25.txt'
gruModelH = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/gru-99-07-25.h5'

bilstmModelTxt = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/bilstm-99-07-25.txt'
bilstmModelH = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/bilstm-99-07-25.h5'

conv1ModelTxt = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/conv1-99-07-25.txt'
conv1ModelH = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/conv1-99-07-25.h5'

CLModelTxt = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/Cl-99-07-25.txt'
CLModelH = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/Cl-99-07-25.h5'

#This file includes tweets and topic number
tweetsTopics = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/Tweets_Topics_990725.csv'



#mr8Similar = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/JokerMostSimilar-99-07-26.csv'
mr8Similar = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/Topics_8_MostSimilar.csv'
mr20Similar = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/Topics_20_MostSimilar.csv'
mr40Similar = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/Topics_40_MostSimilar.csv'
mr80Similar = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/Topics_80_MostSimilar.csv'






#allTrainingData = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/M-A-S140-Processed-98-11-09-03.csv'
#allTrainingData = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/movie-pang-processed-98-12-08.csv'

allTrainingData = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/Processed-Merged-Dataset-99-08-28.csv'
pangLabelledDataset = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/MovieReview-Preprocessed-Pang-Ver2-Labelled.csv'

##Hyper parameters

In [ ]:
#Making Word2vec
embeddingDim = 500
maxLen = embeddingDim
testSize=0.2
windowSize=5
minCount=1 
iterCount=10
sG = 0 #skip gram wil be used


#Making Model
validationSplit = 0.2
#maxFeatures = numWords
maxLen = 500
embeddingSize = 500
isTrainable = False
batchSize = 1000



epochsNum = 50

unitsNum = 20


# Convolution
kernelSize = 3

filtersNum = 500


poolSize = 2
stridesNum = 1
hidden_dims = 100

# LSTM
lstm_output_size = 70

#Login to Google Drive

In [ ]:
#No need in local
from google.colab import drive
drive.mount('/MYDRIVE', force_remount=True)

# Sentiment Analysis by W2V Model

## Loading Data to make w2v (Movies)

## MR8

In [ ]:
dfMR8 = pd.DataFrame()
dfMR8 = pd.read_csv(mr8Similar, encoding = 'utf-8')
print(dfMR8.head(3))
dfMR81 = dfMR8
print(len(dfMR81))

print(dfMR81.columns)

dfMR81.columns = [ 'i', 'docid','Probability']

print(dfMR81.columns)

print(dfMR81.head(3))

print(len(dfMR81))

In [ ]:
len(dfMR81['docid'].unique())

## Loading all Training labelled data

In [ ]:
allTrainingData

In [ ]:
dfALLTraining = pd.DataFrame()
dfALLTraining = pd.read_csv(allTrainingData, encoding = 'utf-8', header=None)
print(dfALLTraining.head(3))
print(len(dfALLTraining))

## Finding Moview docs from Training Data

In [ ]:
len(dfMR81)

In [ ]:
#This data is similar to topic 11
dfTrainingMovies = pd.DataFrame()

dfTrainingMovies = dfALLTraining.loc[dfALLTraining[0].isin(dfMR81['docid'])]
print(len(dfTrainingMovies))
print(dfTrainingMovies.head(3))


## Making Word2Vec Model of movies

In [ ]:
#embeddingDim = 100
#Remove repeated rows
dfMoviesNoRep = dfTrainingMovies
dfMoviesNoRep.columns = ['i','TweetId', 'Tweet', 'Sentiment']
print(dfMoviesNoRep.head(3))  
dfMoviesNoRep.sort_values(["TweetId"], inplace = True) 
print(dfMoviesNoRep.head(3))  
# dropping ALL duplicte values 
dfMoviesNoRep.drop_duplicates(subset ="TweetId", 
                     keep = 'first', inplace = True) 
print(len(dfMoviesNoRep))

In [ ]:
print(len(dfMoviesNoRep[dfMoviesNoRep['Sentiment'] == 0]))

In [ ]:
print(len(dfMoviesNoRep[dfMoviesNoRep['Sentiment'] == 1]))

In [ ]:
# Making a balance dataset with equal number of positive and negative tweets
df1 = dfMoviesNoRep[dfMoviesNoRep['Sentiment'] == 1].sample(73178)
df2 = dfMoviesNoRep[dfMoviesNoRep['Sentiment'] == 0].sample(73178)
df = df1.append(df2)
print(len(df[df['Sentiment'] == 1]))
print(df.head())
print(len(df))

In [ ]:
df.head()

In [ ]:
dfPang = pd.DataFrame()
dfPang = pd.read_csv(pangLabelledDataset, encoding = 'utf-8', header=None)
print(dfPang.head(3))
print(len(dfPang))
dfPang.insert(0,'i',0)
dfPang.columns = ['i' , 'TweetId', 'Tweet', 'Sentiment']
df = df.append(dfPang)
print(len(df))
print(len(df[df['Sentiment'] == 1]))
print(len(df[df['Sentiment'] == 0]))

In [ ]:
dfPang.head(3)

In [ ]:
df.head()

In [ ]:
from sklearn.model_selection import train_test_split
x_train_m, x_test_m, y_train_m, y_test_m = train_test_split( df['Tweet'], df['Sentiment'], test_size=0.2, random_state=10)


In [ ]:
print(len(x_test_m))
print(len(x_train_m))

In [ ]:
print(len(y_test_m[y_test_m == 0]))
print(len(y_test_m[y_test_m == 1]))

In [ ]:
print(len(y_train_m[y_train_m == 0]))
print(len(y_train_m[y_train_m == 1]))

In [ ]:
reviewMovieLines = list()
for line in x_train_m:
  reviewMovieLines.append( str(line).split() )

for line in x_test_m:
  reviewMovieLines.append( str(line).split() )

print(len(reviewMovieLines))

In [ ]:
'''
reviewMovieLines = list()

lines = df['Tweet'].values.tolist()
print(str(lines[0]).split())
for line in lines:
  
  reviewMovieLines.append( str(line).split() )
  '''

In [ ]:
import gensim


w2vMovie = gensim.models.Word2Vec( sentences=reviewMovieLines, size=embeddingDim,  window=windowSize, workers=4, min_count=minCount, iter=iterCount, sg = sG )

w2vGensimModel = gensim.models.Word2Vec()
#vocab size

words = list( w2vMovie.wv.vocab )

print('Vocabulary size: %d' % len(words))

In [ ]:
w2vMovieTxt

In [ ]:
w2vMovie.wv.save_word2vec_format( w2vMovieTxt, binary = False)
w2vMovie.save(w2vMovieM)

In [ ]:
w2vMovie.wv.most_similar('film')

In [ ]:
w2vMovie.wv.most_similar('movi')

## Desining new network by w2v

In [ ]:
import os
embeddingsIndex = {}
f = open(os.path.join('', w2vMovieTxt ), encoding = "utf-8")

for line in f:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:])
  embeddingsIndex[word] = coefs
f.close()

In [ ]:
tokenizerObj = Tokenizer()
tokenizerObj.fit_on_texts(reviewMovieLines)
sequences = tokenizerObj.texts_to_sequences(reviewMovieLines)

print(type(sequences))

reviewPad = pad_sequences( sequences, maxlen=maxLen)

wordIndex = tokenizerObj.word_index
print('Found %s unique tokens.' % len(wordIndex))

#reviewPad = pad_sequences( sequences, maxlen=maxLen)
#sentiment = df ['Sentiment'].values
print(type(reviewPad))
print(len(reviewPad))

x_train_pad = reviewPad[ : len(x_train_m)]
x_test_pad = reviewPad[ len(x_train_m) : ]
y_train = y_train_m
y_test = y_test_m

print(type(x_train_pad))

print('Shape of x_train_pad tensor:' , len(x_train_pad))
print('Shape of y_train tensor:' , len(y_train))


print('Shape of x_test_pad tensor:' , len(x_test_pad))
print('Shape of y_test tensor:' , len( y_test ))


In [ ]:

'''
trainTemp = list()
testTemp = list()

for line in x_test_m:
  testTemp.append( str(line).split() )

for line in x_train_m:
  trainTemp.append( str(line).split() )
  

tokenizerObj1 = Tokenizer()
tokenizerObj1.fit_on_texts(testTemp)
seqTest = tokenizerObj1.texts_to_sequences(testTemp)

wordIndex = tokenizerObj1.word_index
print('Found %s unique tokens.' % len(wordIndex))

tokenizerObj1 = Tokenizer()
tokenizerObj1.fit_on_texts(trainTemp)
seqTrain = tokenizerObj1.texts_to_sequences(trainTemp)


wordIndex = tokenizerObj1.word_index
print('Found %s unique tokens.' % len(wordIndex))


x_test_pad = pad_sequences( seqTest , maxlen=maxLen)
x_train_pad = pad_sequences( seqTrain , maxlen=maxLen)

y_train = y_train_m
y_test = y_test_m



print('Shape of x_train_pad tensor:' , x_train_pad.shape)
print('Shape of y_train tensor:' , y_train.shape)


print('Shape of x_test_pad tensor:' , x_test_pad.shape)
print('Shape of y_test tensor:' , y_test.shape)
'''

In [ ]:
x = y_test[y_test == 0]
print(len(x))

In [ ]:
x = y_test[y_test == 1]
print(len(x))

In [ ]:
x = y_train[y_train == 0]
print(len(x))
x = y_train[y_train == 1]
print(len(x))

In [ ]:
embeddingDim

In [ ]:
numWords = len(wordIndex) + 1
print(numWords)
embeddingMatrix = np.zeros( (numWords, embeddingDim) )
#print(embeddingMatrix.shape)
for word, i in wordIndex.items():
  if i > numWords:
    continue
  embeddingVector = embeddingsIndex.get(word)
  if embeddingVector is not None:
    # words not found in embedding index will be all-zeros.
    embeddingMatrix[i] = embeddingVector

In [ ]:
print(embeddingMatrix.shape)

In [ ]:
max_features = numWords

# Finding Tweets relevant to topic number

In [ ]:
dfTweets = pd.read_csv(tweetsTopics)
print (tweetsTopics)

In [ ]:
dfTweetsRelevantTopic =  dfTweets[dfTweets['Dominant_Topic'] == topicNumber] 

In [ ]:
len(dfTweetsRelevantTopic)




# CNN+BIGRU


In [ ]:
# Embedding

max_features = numWords




# Convolution

print('Build model...')

modelCBIGRU = Sequential(name='CNNBIGRU')

modelCBIGRU.add(Embedding(numWords, embeddingDim , 
                      embeddings_initializer = Constant( embeddingMatrix), 
                      input_length=maxLen,
                      trainable = isTrainable))


modelCBIGRU.add(Dropout(0.3))

modelCBIGRU.add(Conv1D(filters= filtersNum, kernel_size= 3, strides= stridesNum, padding='same', activation='relu'))
modelCBIGRU.add(MaxPooling1D(pool_size = poolSize))

modelCBIGRU.add(Dropout(0.5))
modelCBIGRU.add(Bidirectional(GRU(20, dropout=0.5 )))
modelCBIGRU.add(BatchNormalization())
modelCBIGRU.add(Dropout(0.5))


modelCBIGRU.add(Dense(1,activation='sigmoid'))



opt = SGD(lr=0.1)


modelCBIGRU.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


#modelCGRU.summary()

print('Train...')

rlrp = ReduceLROnPlateau(monitor='val_loss', 
                                  factor=0.2, 
                                  patience=4, 
                                  verbose=1, 
                                  mode='auto', 
                                  min_delta=0.0001, 
                                  cooldown=8, 
                                  min_lr=0)




 
historyCBIGRU = modelCBIGRU.fit(x_train_pad, y_train,
          batch_size=batchSize,
          epochs=epochsNum,
          validation_data=(x_test_pad, y_test), verbose = 1 )#, callbacks=[rlrp])






In [ ]:
testLossCBIGRU, testAccCBIGRU = modelCBIGRU.evaluate(x_test_pad, y_test, batch_size=batchSize, verbose=1)
print('Test loss:', testLossCBIGRU)
print('Test accuracy:', testAccCBIGRU)

trainLossCBIGRU, trainAccCBIGRU = modelCBIGRU.evaluate(x_train_pad, y_train, batch_size=batchSize, verbose=1)
print('Train loss:', trainLossCBIGRU)
print('Train accuracy:', trainAccCBIGRU)

In [ ]:
modelCBIGRU.summary()

In [ ]:
plt.title('Accuracy')
plt.plot(historyCBIGRU.history['accuracy'], label='train')
plt.plot(historyCBIGRU.history['val_accuracy'], label='test')
plt.legend()
plt.show();

In [ ]:
plt.title('Loss')
plt.plot(historyCBIGRU.history['loss'], label='train')
plt.plot(historyCBIGRU.history['val_loss'], label='test')
plt.legend()
plt.show();

In [ ]:
y_val_pred_cbg=modelCBIGRU.predict_classes(x_test_pad)
precisionCBIGRU, recallCBIGRU, fscoreCBIGRU, supportCBIGRU = score(y_test, y_val_pred_cbg)

print('precision: {}'.format(precisionCBIGRU))
print('recall: {}'.format(recallCBIGRU))
print('fscore: {}'.format(fscoreCBIGRU))
print('support: {}'.format(supportCBIGRU))




In [ ]:
from sklearn.metrics import cohen_kappa_score

cohen_score = cohen_kappa_score(y_test, y_val_pred_cbg)

print(cohen_score)

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
labels = [0,1]
def cm_analysis(y_true, y_pred, labels, ymap=None, figsize=(10,10)):
    """
    Generate matrix plot of confusion matrix with pretty annotations.
    The plot image is saved to disk.
    args: 
      y_true:    true label of the data, with shape (nsamples,)
      y_pred:    prediction of the data, with shape (nsamples,)
      filename:  filename of figure file to save
      labels:    string array, name the order of class labels in the confusion matrix.
                 use `clf.classes_` if using scikit-learn models.
                 with shape (nclass,).
      ymap:      dict: any -> string, length == nclass.
                 if not None, map the labels & ys to more understandable strings.
                 Caution: original y_true, y_pred and labels must align.
      figsize:   the size of the figure plotted.
    """
    if ymap is not None:
        y_pred = [ymap[yi] for yi in y_pred]
        y_true = [ymap[yi] for yi in y_true]
        labels = [ymap[yi] for yi in labels]
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.1f%%\n%d/%d' % (p, c, s)
            elif c == 0:
                annot[i, j] = ''
            else:
                annot[i, j] = '%.1f%%\n%d' % (p, c)
    cm = pd.DataFrame(cm, index=labels, columns=labels)
    cm.index.name = 'Actual'
    cm.columns.name = 'Predicted'
    #fig, ax = plt.subplots(figsize=figsize)
    fig, ax = plt.subplots()
    sns.heatmap(cm, annot=annot, fmt='', ax=ax)
    #plt.savefig(filename)
    plt.show()

cm_analysis(y_test, y_val_pred_cbg, labels, ymap=None, figsize=(5,5))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt     
from sklearn.metrics import confusion_matrix


labels = [0,1]
cm = confusion_matrix(y_test, y_val_pred_cbg, labels)

ax= plt.subplot()
sns.heatmap(cm, annot=True, fmt="g", ax = ax,); #annot=True to annotate cells

# labels, title and ticks
ax.set_xlabel('Predicted labels');
ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(labels); 
ax.yaxis.set_ticklabels(labels);

In [ ]:
'''
print(dfTweetsRelevantTopic.head(3))
#testJokerSamples = dfJokerTweets[1]
testJokerSamples = dfTweetsRelevantTopic["Text"]
testJokerSamples = testJokerSamples.dropna()
#print(testSamples[1])
tokenizerObj.fit_on_texts(testJokerSamples)
testJokerSamplesTokens = tokenizerObj.texts_to_sequences( testJokerSamples )
testJokerSamplesTokensPad = pad_sequences( testJokerSamplesTokens, maxlen = maxLen)
print(len(testJokerSamplesTokensPad))

Xnew = testJokerSamplesTokensPad



ynew = modelCBIGRU.predict_classes(Xnew)
#ynew = modelCGRU.predict_classes(Xnew)
#ynew = modelCGRU.predict_classes(Xnew)
print(type(ynew))
print(ynew[0])


numPositiveCBIGRU = 0
numNegativeCBIGRU = 0
for i in range(0, len(ynew)):
  if ynew[i]==0:
    numNegativeCBIGRU = numNegativeCBIGRU + 1
  elif  ynew[i] == 1:
    numPositiveCBIGRU = numPositiveCBIGRU + 1
print('Positive', numPositiveCBIGRU)
print('Negatives', numNegativeCBIGRU)
'''

# CNN+GRU


In [ ]:
import numpy as np
import tensorflow as tf

np.random.seed(1234)
tf.random.set_seed(1234)

In [ ]:
from keras.layers import GlobalMaxPooling1D

max_features = numWords
epochsNum = 20
batchSize = 500
poolSize = 2
stridesNum = 1
max_features = numWords
print('Build model...')
modelCGRU = Sequential(name='CNNGRU')
modelCGRU.add(Embedding(numWords, embeddingDim , 
                      embeddings_initializer = Constant( embeddingMatrix), 
                      input_length=maxLen,
                      trainable = isTrainable))

modelCGRU.add(Conv1D(filters= 500, kernel_size= 7, strides= stridesNum, padding='same', activation='relu'))
#modelCGRU.add(BatchNormalization()) 
modelCGRU.add(Dropout(0.4)) 
modelCGRU.add(MaxPooling1D(pool_size = poolSize))
#modelCGRU.add(Dropout(0.3))
modelCGRU.add(GRU(20, dropout=0.6))
#modelCGRU.add(Dropout(0.3))
modelCGRU.add(Dense(10, activation='relu'))
modelCGRU.add(Dense(1,activation='sigmoid'))





opt = SGD(lr=0.1)
modelCGRU.compile(loss='binary_crossentropy',
              optimizer= 'adam',
              metrics=['accuracy'])
print('Train...')
rlrp = ReduceLROnPlateau(monitor='val_loss', 
                                  factor=0.2, 
                                  patience=4, 
                                  verbose=1, 
                                  mode='auto', 
                                  min_delta=0.0001, 
                                  cooldown=8, 
                                  min_lr=0)

historyCGRU = modelCGRU.fit(x_train_pad, y_train,
          batch_size=batchSize,
          epochs=epochsNum,
          validation_data=(x_test_pad, y_test), verbose = 1 )#, callbacks=[rlrp])
clear_output()


In [ ]:
testLossCGRU, testAccCGRU = modelCGRU.evaluate(x_test_pad, y_test, batch_size=batchSize, verbose=1)
print('Test loss:', testLossCGRU)
print('Test accuracy:', testAccCGRU)

trainLossCGRU, trainAccCGRU = modelCGRU.evaluate(x_train_pad, y_train, batch_size=batchSize, verbose=1)
print('Train loss:', trainLossCGRU)
print('Train accuracy:', trainAccCGRU)

In [ ]:
y_val_pred_cgru=modelCGRU.predict_classes(x_test_pad)
precisionCGRU, recallCGRU, fscoreCGRU, supportCGRU = score(y_test, y_val_pred_cgru)

print('precision: {}'.format(precisionCGRU))
print('recall: {}'.format(recallCGRU))
print('fscore: {}'.format(fscoreCGRU))
print('support: {}'.format(supportCGRU))

In [ ]:
plt.title('Accuracy')
plt.plot(historyCGRU.history['accuracy'], label='train')
plt.plot(historyCGRU.history['val_accuracy'], label='test')
plt.legend()
plt.show();

In [ ]:
plt.title('Loss')
plt.plot(historyCGRU.history['loss'], label='train')
plt.plot(historyCGRU.history['val_loss'], label='test')
plt.legend()
plt.show();

In [ ]:
y_val_pred_cgru=modelCGRU.predict_classes(x_test_pad)
precisionCGRU, recallCGRU, fscoreCGRU, supportCGRU = score(y_test, y_val_pred_cgru)

print('precision: {}'.format(precisionCGRU))
print('recall: {}'.format(recallCGRU))
print('fscore: {}'.format(fscoreCGRU))
print('support: {}'.format(supportCGRU))


In [ ]:
from sklearn.metrics import cohen_kappa_score
cohen_score = cohen_kappa_score(y_test, y_val_pred)
print(cohen_score)

In [ ]:
len(y_test[y_test==1])

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt     
from sklearn.metrics import confusion_matrix

labels = [0,1]
cm = confusion_matrix(y_test, y_val_pred_cgru, labels)

ax= plt.subplot()
sns.heatmap(cm, annot=True, fmt="g", ax = ax,); #annot=True to annotate cells

# labels, title and ticks
ax.set_xlabel('Predicted labels');
ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(labels); 
ax.yaxis.set_ticklabels(labels);

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
labels = [0,1]
def cm_analysis(y_true, y_pred, labels, ymap=None, figsize=(10,10)):
    """
    Generate matrix plot of confusion matrix with pretty annotations.
    The plot image is saved to disk.
    args: 
      y_true:    true label of the data, with shape (nsamples,)
      y_pred:    prediction of the data, with shape (nsamples,)
      filename:  filename of figure file to save
      labels:    string array, name the order of class labels in the confusion matrix.
                 use `clf.classes_` if using scikit-learn models.
                 with shape (nclass,).
      ymap:      dict: any -> string, length == nclass.
                 if not None, map the labels & ys to more understandable strings.
                 Caution: original y_true, y_pred and labels must align.
      figsize:   the size of the figure plotted.
    """
    if ymap is not None:
        y_pred = [ymap[yi] for yi in y_pred]
        y_true = [ymap[yi] for yi in y_true]
        labels = [ymap[yi] for yi in labels]
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.1f%%\n%d/%d' % (p, c, s)
            elif c == 0:
                annot[i, j] = ''
            else:
                annot[i, j] = '%.1f%%\n%d' % (p, c)
    cm = pd.DataFrame(cm, index=labels, columns=labels)
    cm.index.name = 'Actual'
    cm.columns.name = 'Predicted'
    #fig, ax = plt.subplots(figsize=figsize)
    fig, ax = plt.subplots()
    sns.heatmap(cm, annot=annot, fmt='', ax=ax)
    #plt.savefig(filename)
    plt.show()

cm_analysis(y_test, y_val_pred_cgru, labels, ymap=None, figsize=(5,5))

In [ ]:
'''
print(dfTweetsRelevantTopic.head(3))
#testJokerSamples = dfJokerTweets[1]
testJokerSamples = dfTweetsRelevantTopic["Text"]
testJokerSamples = testJokerSamples.dropna()
#print(testSamples[1])
tokenizerObj.fit_on_texts(testJokerSamples)
testJokerSamplesTokens = tokenizerObj.texts_to_sequences( testJokerSamples )
testJokerSamplesTokensPad = pad_sequences( testJokerSamplesTokens, maxlen = maxLen)
print(len(testJokerSamplesTokensPad))

Xnew = testJokerSamplesTokensPad



ynew = modelCGRU_s.predict_classes(Xnew)
#ynew = modelCGRU.predict_classes(Xnew)
#ynew = modelCGRU.predict_classes(Xnew)
print(type(ynew))
print(ynew[0])


numPositiveCGRU = 0
numNegativeCGRU = 0
for i in range(0, len(ynew)):
  if ynew[i]==0:
    numNegativeCGRU = numNegativeCGRU + 1
  elif  ynew[i] == 1:
    numPositiveCGRU = numPositiveCGRU + 1
print('Positive', numPositiveCGRU)
print('Negatives', numNegativeCGRU)
'''

# CNN+BiLSTM

In [ ]:
# Embedding

max_features = numWords
filtersNum = 400
unitsNum = 20

# Convolution




print('Build model...')

modelCBiLSTM = Sequential(name='CNNBiLSTM')
modelCBiLSTM.add(Embedding(numWords, embeddingDim , 
                      embeddings_initializer = Constant( embeddingMatrix), 
                      input_length=maxLen,
                      trainable = isTrainable))


modelCBiLSTM.add(Conv1D(filters= filtersNum, kernel_size= kernelSize, strides= stridesNum, padding='same', activation='relu'))


modelCBiLSTM.add(Dropout(0.5))
modelCBiLSTM.add(MaxPooling1D(pool_size = poolSize))

modelCBiLSTM.add(Dropout(0.5))
modelCBiLSTM.add(Bidirectional(LSTM(20, dropout=0.5 )))
modelCBiLSTM.add(BatchNormalization())
modelCBiLSTM.add(Dropout(0.5))

modelCBiLSTM.add(Dense(1,activation='sigmoid'))



opt = SGD(lr=0.1)


modelCBiLSTM.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


#modelCBiLSTM.summary()

print('Train...')

rlrp = ReduceLROnPlateau(monitor='val_loss', 
                                  factor=0.1, 
                                  patience=8, 
                                  verbose=1, 
                                  mode='auto', 
                                  min_delta=0.0001, 
                                  cooldown=8, 
                                  min_lr=0)
#lrm = LearningRateMonitor()
 
historyCBiLSTM = modelCBiLSTM.fit(x_train_pad, y_train,
          batch_size=batchSize,
          epochs=epochsNum,
          validation_data=(x_test_pad, y_test), verbose = 2 )#, callbacks=[rlrp])



In [ ]:
testLossCBiLSTM, testAccCBiLSTM = modelCBiLSTM.evaluate(x_test_pad, y_test, batch_size=batchSize, verbose=1)
print('Test loss:', testLossCBiLSTM)
print('Test accuracy:', testAccCBiLSTM)

In [ ]:
trainLossCBiLSTM, trainAccCBiLSTM = modelCBiLSTM.evaluate(x_train_pad, y_train, batch_size=batchSize, verbose=1)
print('Train loss:', trainLossCBiLSTM)
print('Train accuracy:', trainAccCBiLSTM)

In [ ]:
modelCBiLSTM.summary()

In [ ]:
plt.title('Accuracy')
plt.plot(historyCBiLSTM.history['accuracy'], label='train')
plt.plot(historyCBiLSTM.history['val_accuracy'], label='test')
plt.legend()
plt.show();

In [ ]:
plt.title('Loss')
plt.plot(historyCBiLSTM.history['loss'], label='train')
plt.plot(historyCBiLSTM.history['val_loss'], label='test')
plt.legend()
plt.show();

In [ ]:
y_val_pred_cbl = modelCBiLSTM.predict_classes(x_test_pad)
precisionCBiLSTM, recallCBiLSTM, fscoreCBiLSTM, supportCBiLSTM = score(y_test, y_val_pred_cbl)

print('precision: {}'.format(precisionCBiLSTM))
print('recall: {}'.format(recallCBiLSTM))
print('fscore: {}'.format(fscoreCBiLSTM))
print('support: {}'.format(supportCBiLSTM))




In [ ]:
from sklearn.metrics import cohen_kappa_score

cohen_score = cohen_kappa_score(y_test, y_val_pred_cbl)

print(cohen_score)

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
labels = [0,1]
def cm_analysis(y_true, y_pred, labels, ymap=None, figsize=(10,10)):
    """
    Generate matrix plot of confusion matrix with pretty annotations.
    The plot image is saved to disk.
    args: 
      y_true:    true label of the data, with shape (nsamples,)
      y_pred:    prediction of the data, with shape (nsamples,)
      filename:  filename of figure file to save
      labels:    string array, name the order of class labels in the confusion matrix.
                 use `clf.classes_` if using scikit-learn models.
                 with shape (nclass,).
      ymap:      dict: any -> string, length == nclass.
                 if not None, map the labels & ys to more understandable strings.
                 Caution: original y_true, y_pred and labels must align.
      figsize:   the size of the figure plotted.
    """
    if ymap is not None:
        y_pred = [ymap[yi] for yi in y_pred]
        y_true = [ymap[yi] for yi in y_true]
        labels = [ymap[yi] for yi in labels]
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.1f%%\n%d/%d' % (p, c, s)
            elif c == 0:
                annot[i, j] = ''
            else:
                annot[i, j] = '%.1f%%\n%d' % (p, c)
    cm = pd.DataFrame(cm, index=labels, columns=labels)
    cm.index.name = 'Actual'
    cm.columns.name = 'Predicted'
    fig, ax = plt.subplots(figsize=figsize)
    sns.heatmap(cm, annot=annot, fmt='', ax=ax)
    #plt.savefig(filename)
    plt.show()

cm_analysis(y_test, y_val_pred_cbl, labels, ymap=None, figsize=(5,5))



In [ ]:
'''
print(dfTweetsRelevantTopic.head(3))
#testJokerSamples = dfJokerTweets[1]
testJokerSamples = dfTweetsRelevantTopic["Text"]
testJokerSamples = testJokerSamples.dropna()
#print(testSamples[1])
tokenizerObj.fit_on_texts(testJokerSamples)
testJokerSamplesTokens = tokenizerObj.texts_to_sequences( testJokerSamples )
testJokerSamplesTokensPad = pad_sequences( testJokerSamplesTokens, maxlen = maxLen)
print(len(testJokerSamplesTokensPad))

Xnew = testJokerSamplesTokensPad
ynew = modelCBiLSTM.predict_classes(Xnew)
print(type(ynew))
print(ynew[0])


numPositiveCBiLSTM = 0
numNegativeCBiLSTM = 0
for i in range(0, len(ynew)):
  if ynew[i]==0:
    numNegativeCBiLSTM = numNegativeCBiLSTM + 1
  elif  ynew[i] == 1:
    numPositiveCBiLSTM = numPositiveCBiLSTM + 1
print('Positive', numPositiveCBiLSTM)
print('Negatives', numNegativeCBiLSTM)
'''

# CNN+LSTM

In [ ]:
# Embedding



max_features = numWords


filtersNum = 500


unitsNum = 20


# Convolution



'''
Note:
batch_size is highly sensitive.
Only 2 epochs are needed as the dataset is very small.
'''
print('Build model...')

modelCL = Sequential(name='CNNLSTM')
modelCL.add(Embedding(numWords, embeddingDim , 
                      embeddings_initializer = Constant( embeddingMatrix), 
                      input_length=maxLen,
                      trainable = isTrainable))


modelCL.add(Conv1D(filters= filtersNum, kernel_size= kernelSize, strides= stridesNum, padding='same', activation='relu'))


modelCL.add(Dropout(0.5))
modelCL.add(MaxPooling1D(pool_size = poolSize))

modelCL.add(Dropout(0.5))
modelCL.add(LSTM(20, dropout=0.5 ))
modelCL.add(BatchNormalization())
modelCL.add(Dropout(0.5))



modelCL.add(Dense(1,activation='sigmoid'))


opt = SGD(lr=0.1)


modelCL.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


#modelCL.summary()

print('Train...')

rlrp = ReduceLROnPlateau(monitor='val_loss', 
                                  factor=0.1, 
                                  patience=8, 
                                  verbose=1, 
                                  mode='auto', 
                                  min_delta=0.0001, 
                                  cooldown=8, 
                                  min_lr=0)
#lrm = LearningRateMonitor()
 
historyCL = modelCL.fit(x_train_pad, y_train,
          batch_size=batchSize,
          epochs=epochsNum,
          validation_data=(x_test_pad, y_test), verbose = 1 )#, callbacks=[rlrp])


In [ ]:
testLossCL, testAccCL = modelCL.evaluate(x_test_pad, y_test, batch_size=batchSize, verbose=1)
print('Test loss:', testLossCL)
print('Test accuracy:', testAccCL)

In [ ]:
trainLossCL, trainAccCL = modelCL.evaluate(x_train_pad, y_train, batch_size=batchSize, verbose=1)
print('Train loss:', trainLossCL)
print('Train accuracy:', trainAccCL)

In [ ]:
#modelCL.save(CLModelTxt)
#modelCL.save(CLModelH)


In [ ]:
modelCL.summary()

In [ ]:
plt.title('Accuracy')
plt.plot(historyCL.history['accuracy'], label='train')
plt.plot(historyCL.history['val_accuracy'], label='test')
plt.legend()
plt.show();

In [ ]:
plt.title('Loss')
plt.plot(historyCL.history['loss'], label='train')
plt.plot(historyCL.history['val_loss'], label='test')
plt.legend()
plt.show();

In [ ]:
y_val_pred_cl=modelCL.predict_classes(x_test_pad)
precisionCL, recallCL, fscoreCL, supportCL = score(y_test, y_val_pred_cl)

print('precision: {}'.format(precisionCL))
print('recall: {}'.format(recallCL))
print('fscore: {}'.format(fscoreCL))
print('support: {}'.format(supportCL))

In [ ]:
from sklearn.metrics import cohen_kappa_score

cohen_score = cohen_kappa_score(y_test, y_val_pred_cl)

print(cohen_score)

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
labels = [0,1]
def cm_analysis(y_true, y_pred, labels, ymap=None, figsize=(10,10)):
    """
    Generate matrix plot of confusion matrix with pretty annotations.
    The plot image is saved to disk.
    args: 
      y_true:    true label of the data, with shape (nsamples,)
      y_pred:    prediction of the data, with shape (nsamples,)
      filename:  filename of figure file to save
      labels:    string array, name the order of class labels in the confusion matrix.
                 use `clf.classes_` if using scikit-learn models.
                 with shape (nclass,).
      ymap:      dict: any -> string, length == nclass.
                 if not None, map the labels & ys to more understandable strings.
                 Caution: original y_true, y_pred and labels must align.
      figsize:   the size of the figure plotted.
    """
    if ymap is not None:
        y_pred = [ymap[yi] for yi in y_pred]
        y_true = [ymap[yi] for yi in y_true]
        labels = [ymap[yi] for yi in labels]
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.1f%%\n%d/%d' % (p, c, s)
            elif c == 0:
                annot[i, j] = ''
            else:
                annot[i, j] = '%.1f%%\n%d' % (p, c)
    cm = pd.DataFrame(cm, index=labels, columns=labels)
    cm.index.name = 'Actual'
    cm.columns.name = 'Predicted'
    fig, ax = plt.subplots(figsize=figsize)
    sns.heatmap(cm, annot=annot, fmt='', ax=ax)
    #plt.savefig(filename)
    plt.show()

cm_analysis(y_test, y_val_pred_cl, labels, ymap=None, figsize=(5,5))



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt     
from sklearn.metrics import confusion_matrix


labels = [0,1]
cm = confusion_matrix(y_test, y_val_pred_cl, labels)

ax= plt.subplot()
sns.heatmap(cm, annot=True, fmt="g", ax = ax,); #annot=True to annotate cells

# labels, title and ticks
ax.set_xlabel('Predicted labels');
ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(labels); 
ax.yaxis.set_ticklabels(labels);


## Predicting Joker

*Here predicting will be done based on one topic*

In [ ]:
'''
print(dfTweetsRelevantTopic.head(3))
#testJokerSamples = dfJokerTweets[1]
testJokerSamples = dfTweetsRelevantTopic["Text"]
testJokerSamples = testJokerSamples.dropna()
#print(testSamples[1])
tokenizerObj.fit_on_texts(testJokerSamples)
testJokerSamplesTokens = tokenizerObj.texts_to_sequences( testJokerSamples )
testJokerSamplesTokensPad = pad_sequences( testJokerSamplesTokens, maxlen = maxLen)
print(len(testJokerSamplesTokensPad))

Xnew = testJokerSamplesTokensPad
ynew = modelCL.predict_classes(Xnew)
print(type(ynew))
print(ynew[0])


numPositiveCL = 0
numNegativeCL = 0
for i in range(0, len(ynew)):
  if ynew[i]==0:
    numNegativeCL = numNegativeCL + 1
  elif  ynew[i] == 1:
    numPositiveCL = numPositiveCL + 1
print('Positive', numPositiveCL)
print('Negatives', numNegativeCL)
'''

# LSTM

In [ ]:
max_features = numWords

# LSTM
lstm_output_size = 70

unitsNum = 10
batchSize = 32



opt = SGD(lr=0.01, momentum=0.9)
#
print('Build model...')
modelLSTM = Sequential()



lstm_out = 196


embeddingLayer = Embedding( numWords,
                           embeddingDim,
                           embeddings_initializer = Constant( embeddingMatrix),
                           input_length = maxLen,
                           trainable = isTrainable)
modelLSTM.add(embeddingLayer)
modelLSTM.add(Dropout(0.5))
modelLSTM.add(LSTM(unitsNum, dropout=0.5))

modelLSTM.add(Dropout(0.5))

modelLSTM.add(Dense(1,activation='sigmoid'))
#modelLSTM.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

opt = SGD(lr=0.1)
modelLSTM.compile(loss='binary_crossentropy',optimizer=opt, metrics=['accuracy'])



#modelLSTM.summary()

rlrp = ReduceLROnPlateau(monitor='val_loss', 
                                  factor=0.1, 
                                  patience=8, 
                                  verbose=1, 
                                  mode='auto', 
                                  min_delta=0.0001, 
                                  cooldown=8, 
                                  min_lr=0)

historyLSTM = modelLSTM.fit(x_train_pad, y_train,
          batch_size=batchSize,
          epochs=epochsNum,
          validation_data=(x_test_pad, y_test),  verbose = 1)#, callbacks=[rlrp])



In [ ]:
testLossLSTM, testAccLSTM = modelLSTM.evaluate(x_test_pad, y_test,
                            batch_size=batchSize)
print('Test loss:', testLossLSTM)
print('Test accuracy:', testAccLSTM)

In [ ]:
trainLossLSTM, trainAccLSTM = modelLSTM.evaluate(x_train_pad, y_train, batch_size=batchSize, verbose=1)
print('Train loss:', trainLossLSTM)
print('Train accuracy:', trainAccLSTM)

In [ ]:
#modelLSTM.save(lstmModelTxt)
#modelLSTM.save(lstmModelH)

In [ ]:
modelLSTM.summary()

In [ ]:
plt.title('Accuracy')
plt.plot(historyLSTM.history['accuracy'], label='train')
plt.plot(historyLSTM.history['val_accuracy'], label='test')
plt.legend()
plt.show();

In [ ]:
plt.title('Loss')
plt.plot(historyLSTM.history['loss'], label='train')
plt.plot(historyLSTM.history['val_loss'], label='test')
plt.legend()
plt.show();

In [ ]:
y_val_pred_l=modelLSTM.predict_classes(x_test_pad)
precisionLSTM, recallLSTM, fscoreLSTM, supportLSTM = score(y_test, y_val_pred_l)

print('precision: {}'.format(precisionLSTM))
print('recall: {}'.format(recallLSTM))
print('fscore: {}'.format(fscoreLSTM))
print('support: {}'.format(supportLSTM))

In [ ]:
from sklearn.metrics import cohen_kappa_score

cohen_score = cohen_kappa_score(y_test, y_val_pred_l)

print(cohen_score)

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
labels = [0,1]
def cm_analysis(y_true, y_pred, labels, ymap=None, figsize=(10,10)):
    """
    Generate matrix plot of confusion matrix with pretty annotations.
    The plot image is saved to disk.
    args: 
      y_true:    true label of the data, with shape (nsamples,)
      y_pred:    prediction of the data, with shape (nsamples,)
      filename:  filename of figure file to save
      labels:    string array, name the order of class labels in the confusion matrix.
                 use `clf.classes_` if using scikit-learn models.
                 with shape (nclass,).
      ymap:      dict: any -> string, length == nclass.
                 if not None, map the labels & ys to more understandable strings.
                 Caution: original y_true, y_pred and labels must align.
      figsize:   the size of the figure plotted.
    """
    if ymap is not None:
        y_pred = [ymap[yi] for yi in y_pred]
        y_true = [ymap[yi] for yi in y_true]
        labels = [ymap[yi] for yi in labels]
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.1f%%\n%d/%d' % (p, c, s)
            elif c == 0:
                annot[i, j] = ''
            else:
                annot[i, j] = '%.1f%%\n%d' % (p, c)
    cm = pd.DataFrame(cm, index=labels, columns=labels)
    cm.index.name = 'Actual'
    cm.columns.name = 'Predicted'
    #fig, ax = plt.subplots(figsize=figsize)
    fig, ax = plt.subplots()
    sns.heatmap(cm, annot=annot, fmt='', ax=ax)
    #plt.savefig(filename)
    plt.show()

cm_analysis(y_test, y_val_pred_l, labels, ymap=None, figsize=(5,5))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt     
from sklearn.metrics import confusion_matrix


labels = [0,1]
cm = confusion_matrix(y_test, y_val_pred_l, labels)

ax= plt.subplot()
sns.heatmap(cm, annot=True, fmt="g", ax = ax,); #annot=True to annotate cells

# labels, title and ticks
ax.set_xlabel('Predicted labels');
ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(labels); 
ax.yaxis.set_ticklabels(labels);


In [ ]:
'''
print(dfTweetsRelevantTopic.head(3))
testJokerSamples = dfTweetsRelevantTopic["Text"]
testJokerSamples = testJokerSamples.dropna()
#print(testSamples[1])
tokenizerObj.fit_on_texts(testJokerSamples)
testJokerSamplesTokens = tokenizerObj.texts_to_sequences( testJokerSamples )
testJokerSamplesTokensPad = pad_sequences( testJokerSamplesTokens, maxlen = maxLen)
print(len(testJokerSamplesTokensPad))

Xnew = testJokerSamplesTokensPad
ynew = modelLSTM.predict_classes(Xnew)
print(type(ynew))
print(ynew[0])


numPositiveLSTM = 0
numNegativeLSTM = 0
for i in range(0, len(ynew)):
  if ynew[i]==0:
    numNegativeLSTM = numNegativeLSTM + 1
  elif  ynew[i] == 1:
    numPositiveLSTM = numPositiveLSTM + 1
print('Positive', numPositiveLSTM)
print('Negatives', numNegativeLSTM)
'''

# GRU

In [ ]:
print('Build model...')


modelGRU = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
embeddingLayer = Embedding( numWords,
                           embeddingDim,
                           embeddings_initializer = Constant( embeddingMatrix),
                           input_length = maxLen,
                           trainable = isTrainable)

modelGRU.add(embeddingLayer)

modelGRU.add(Dropout(0.5))
modelGRU.add(GRU(units= 128 , dropout = 0.5))

modelGRU.add(Dropout(0.5))
modelGRU.add(Dense(1,activation='sigmoid'))


opt = SGD(lr=0.1)

modelGRU.compile(loss='binary_crossentropy',
              optimizer= opt,
              metrics=['accuracy'])

modelGRU.summary()


rlrp = ReduceLROnPlateau(monitor='val_loss', 
                                  factor=0.1, 
                                  patience=8, 
                                  verbose=1, 
                                  mode='auto', 
                                  min_delta=0.0001, 
                                  cooldown=8, 
                                  min_lr=0)

historyGRU = modelGRU.fit(x_train_pad, y_train,
          batch_size=batchSize,
          epochs=100,
          validation_data=(x_test_pad, y_test),
          verbose = 1)  # , callbacks=[rlrp])



In [ ]:
testLossGRU, testAccGRU = modelGRU.evaluate(x_test_pad, y_test,
                            batch_size=batchSize)
print('Test loss:', testLossGRU)
print('Test accuracy:', testAccGRU)

In [ ]:
trainLossGRU, trainAccGRU = modelGRU.evaluate(x_train_pad, y_train,
                            batch_size=batchSize)
print('Train loss:', trainLossGRU)
print('Train accuracy:', trainAccGRU)

In [ ]:
#modelGRU.save(gruModelTxt)
#modelGRU.save(gruModelH)

In [ ]:
modelGRU.summary()

In [ ]:
plt.title('Accuracy')
plt.plot(historyGRU.history['accuracy'], label='train')
plt.plot(historyGRU.history['val_accuracy'], label='test')
plt.legend()
plt.show();

In [ ]:
plt.title('Loss')
plt.plot(historyGRU.history['loss'], label='train')
plt.plot(historyGRU.history['val_loss'], label='test')
plt.legend()
plt.show();

In [ ]:
from sklearn.metrics import precision_recall_fscore_support as score


y_val_pred_g=modelGRU.predict_classes(x_test_pad)
precisionGRU, recallGRU, fscoreGRU, supportGRU = score(y_test, y_val_pred_g)

print('precision: {}'.format(precisionGRU))
print('recall: {}'.format(recallGRU))
print('fscore: {}'.format(fscoreGRU))
print('support: {}'.format(supportGRU))

In [ ]:
from sklearn.metrics import cohen_kappa_score

cohen_score = cohen_kappa_score(y_test, y_val_pred_g)

print(cohen_score)

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
labels = [0,1]
def cm_analysis(y_true, y_pred, labels, ymap=None, figsize=(10,10)):
    """
    Generate matrix plot of confusion matrix with pretty annotations.
    The plot image is saved to disk.
    args: 
      y_true:    true label of the data, with shape (nsamples,)
      y_pred:    prediction of the data, with shape (nsamples,)
      filename:  filename of figure file to save
      labels:    string array, name the order of class labels in the confusion matrix.
                 use `clf.classes_` if using scikit-learn models.
                 with shape (nclass,).
      ymap:      dict: any -> string, length == nclass.
                 if not None, map the labels & ys to more understandable strings.
                 Caution: original y_true, y_pred and labels must align.
      figsize:   the size of the figure plotted.
    """
    if ymap is not None:
        y_pred = [ymap[yi] for yi in y_pred]
        y_true = [ymap[yi] for yi in y_true]
        labels = [ymap[yi] for yi in labels]
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.1f%%\n%d/%d' % (p, c, s)
            elif c == 0:
                annot[i, j] = ''
            else:
                annot[i, j] = '%.1f%%\n%d' % (p, c)
    cm = pd.DataFrame(cm, index=labels, columns=labels)
    cm.index.name = 'Actual'
    cm.columns.name = 'Predicted'
    fig, ax = plt.subplots()
    sns.heatmap(cm, annot=annot, fmt='', ax=ax)
    #plt.savefig(filename)
    plt.show()

cm_analysis(y_test, y_val_pred_g, labels, ymap=None, figsize=(5,5))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt     
from sklearn.metrics import confusion_matrix


labels = [0,1]
cm = confusion_matrix(y_test, y_val_pred_g, labels)

ax= plt.subplot()
sns.heatmap(cm, annot=True, fmt="g", ax = ax,); #annot=True to annotate cells

# labels, title and ticks
ax.set_xlabel('Predicted labels');
ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(labels); 
ax.yaxis.set_ticklabels(labels);


# BIGRU

In [ ]:
print('Build model...')


modelBIGRU = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
embeddingLayer = Embedding( numWords,
                           embeddingDim,
                           embeddings_initializer = Constant( embeddingMatrix),
                           input_length = maxLen,
                           trainable = isTrainable)

modelBIGRU.add(embeddingLayer)
modelBIGRU.add(Dropout(0.5))
modelBIGRU.add(Bidirectional(GRU(units=20, dropout = 0.5)))
modelBIGRU.add(BatchNormalization())

modelBIGRU.add(Dropout(0.5))
modelBIGRU.add(Dense(1,activation='sigmoid'))



modelBIGRU.compile(loss='binary_crossentropy',
              optimizer= 'adam',
              metrics=['accuracy'])


historyBIGRU = modelBIGRU.fit(x_train_pad, y_train,
          batch_size=batchSize,
          epochs=epochsNum,
          validation_data=(x_test_pad, y_test),
          verbose = 1)  # , callbacks=[rlrp])



In [ ]:
testLossBIGRU, testAccBIGRU = modelBIGRU.evaluate(x_test_pad, y_test,
                            batch_size=batchSize)
print('Test loss:', testLossBIGRU)
print('Test accuracy:', testAccBIGRU)

In [ ]:
trainLossBIGRU, trainAccBIGRU = modelBIGRU.evaluate(x_train_pad, y_train,
                            batch_size=batchSize)
print('Train loss:', trainLossBIGRU)
print('Train accuracy:', trainAccBIGRU)

In [ ]:
modelBIGRU.summary()

In [ ]:
plt.title('Accuracy')
plt.plot(historyBIGRU.history['accuracy'], label='train')
plt.plot(historyBIGRU.history['val_accuracy'], label='test')
plt.legend()
plt.show();

In [ ]:
plt.title('Loss')
plt.plot(historyBIGRU.history['loss'], label='train')
plt.plot(historyBIGRU.history['val_loss'], label='test')
plt.legend()
plt.show();

In [ ]:
from sklearn.metrics import precision_recall_fscore_support as score


y_val_pred_bg=modelBIGRU.predict_classes(x_test_pad)
precisionBIGRU, recallBIGRU, fscoreBIGRU, supportBIGRU = score(y_test, y_val_pred_bg)

print('precision: {}'.format(precisionBIGRU))
print('recall: {}'.format(recallBIGRU))
print('fscore: {}'.format(fscoreBIGRU))
print('support: {}'.format(supportBIGRU))

In [ ]:
from sklearn.metrics import cohen_kappa_score

cohen_score = cohen_kappa_score(y_test, y_val_pred_bg)

print(cohen_score)

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
labels = [0,1]
def cm_analysis(y_true, y_pred, labels, ymap=None, figsize=(10,10)):
    """
    Generate matrix plot of confusion matrix with pretty annotations.
    The plot image is saved to disk.
    args: 
      y_true:    true label of the data, with shape (nsamples,)
      y_pred:    prediction of the data, with shape (nsamples,)
      filename:  filename of figure file to save
      labels:    string array, name the order of class labels in the confusion matrix.
                 use `clf.classes_` if using scikit-learn models.
                 with shape (nclass,).
      ymap:      dict: any -> string, length == nclass.
                 if not None, map the labels & ys to more understandable strings.
                 Caution: original y_true, y_pred and labels must align.
      figsize:   the size of the figure plotted.
    """
    if ymap is not None:
        y_pred = [ymap[yi] for yi in y_pred]
        y_true = [ymap[yi] for yi in y_true]
        labels = [ymap[yi] for yi in labels]
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.1f%%\n%d/%d' % (p, c, s)
            elif c == 0:
                annot[i, j] = ''
            else:
                annot[i, j] = '%.1f%%\n%d' % (p, c)
    cm = pd.DataFrame(cm, index=labels, columns=labels)
    cm.index.name = 'Actual'
    cm.columns.name = 'Predicted'
    fig, ax = plt.subplots()
    sns.heatmap(cm, annot=annot, fmt='', ax=ax)
    #plt.savefig(filename)
    plt.show()

cm_analysis(y_test, y_val_pred_bg, labels, ymap=None, figsize=(5,5))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt     
from sklearn.metrics import confusion_matrix


labels = [0,1]
cm = confusion_matrix(y_test, y_val_pred_bg, labels)

ax= plt.subplot()
sns.heatmap(cm, annot=True, fmt="g", ax = ax,); #annot=True to annotate cells

# labels, title and ticks
ax.set_xlabel('Predicted labels');
ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(labels); 
ax.yaxis.set_ticklabels(labels);

## Predicting Joker

In [ ]:
'''
print(dfTweetsRelevantTopic.head(3))
testJokerSamples = dfTweetsRelevantTopic["Text"]
testJokerSamples = testJokerSamples.dropna()
#print(testSamples[1])
tokenizerObj.fit_on_texts(testJokerSamples)
testJokerSamplesTokens = tokenizerObj.texts_to_sequences( testJokerSamples )
testJokerSamplesTokensPad = pad_sequences( testJokerSamplesTokens, maxlen = maxLen)
print(len(testJokerSamplesTokensPad))

Xnew = testJokerSamplesTokensPad
ynew = modelBIGRU.predict_classes(Xnew)
print(type(ynew))
print(ynew[0])


numPositiveBIGRU = 0
numNegativeBIGRU = 0
for i in range(0, len(ynew)):
  if ynew[i]==0:
    numNegativeBIGRU = numNegativeBIGRU + 1
  elif  ynew[i] == 1:
    numPositiveBIGRU = numPositiveBIGRU + 1
print('Positive', numPositiveBIGRU )
print('Negatives', numNegativeBIGRU )
'''


# BILSTM

In [ ]:
maxFeatures = numWords

unitsNumber = 20






opt = SGD(lr=0.01, momentum=0.9)
#model.compile(..., optimizer=opt)
print('Build model...')
modelBiLSTM = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
embeddingLayer = Embedding( numWords,
                           embeddingDim,
                           embeddings_initializer = Constant( embeddingMatrix),
                           input_length = maxLen,
                           trainable = isTrainable)

modelBiLSTM.add(embeddingLayer)

modelBiLSTM.add(Dropout(0.5))

modelBiLSTM.add(Bidirectional(LSTM(unitsNum, dropout = 0.5)))

modelBiLSTM.add(Dropout(0.5))

modelBiLSTM.add(Dense(1,activation='sigmoid'))

modelBiLSTM.compile(loss='binary_crossentropy',optimizer= opt, metrics=['accuracy'])


#modelBiLSTM.summary()

rlrp = ReduceLROnPlateau(monitor='val_loss', 
                                  factor=0.1, 
                                  patience=8, 
                                  verbose=1, 
                                  mode='auto', 
                                  min_delta=0.0001, 
                                  cooldown=8, 
                                  min_lr=0)

historyBILSTM = modelBiLSTM.fit(x_train_pad, y_train,
          batch_size=batchSize,
          epochs=epochsNum,
          validation_data=(x_test_pad, y_test),
          verbose = 1 )#, callbacks=[rlrp])



In [ ]:
testLossBilstm, testAccBilstm = modelBiLSTM.evaluate(x_test_pad, y_test,
                            batch_size=batchSize)
print('Test score:', testLossBilstm)
print('Test accuracy:', testAccBilstm)

In [ ]:
trainLossBilstm, trainAccBilstm = modelBiLSTM.evaluate(x_train_pad, y_train,
                            batch_size=batchSize)
print('Train score:', trainLossBilstm)
print('Train accuracy:', trainAccBilstm)

In [ ]:
#modelBiLSTM.save(bilstmModelTxt)
#modelBiLSTM.save(bilstmModelH)

In [ ]:
modelBiLSTM.summary()

In [ ]:
plt.title('Accuracy')
plt.plot(historyBILSTM.history['accuracy'], label='train')
plt.plot(historyBILSTM.history['val_accuracy'], label='test')
plt.legend()
plt.show();

In [ ]:
plt.title('Loss')
plt.plot(historyBILSTM.history['loss'], label='train')
plt.plot(historyBILSTM.history['val_loss'], label='test')
plt.legend()
plt.show();

In [ ]:
y_val_pred_bl=modelBiLSTM.predict_classes(x_test_pad)
precisionBilstm, recallBilstm, fscoreBilstm, supportBilstm = score(y_test, y_val_pred_bl)

print('precision: {}'.format(precisionBilstm))
print('recall: {}'.format(recallBilstm))
print('fscore: {}'.format(fscoreBilstm))
print('support: {}'.format(supportBilstm))

In [ ]:
from sklearn.metrics import cohen_kappa_score

cohen_score = cohen_kappa_score(y_test, y_val_pred_bl)

print(cohen_score)

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
labels = [0,1]
def cm_analysis(y_true, y_pred, labels, ymap=None, figsize=(10,10)):
    """
    Generate matrix plot of confusion matrix with pretty annotations.
    The plot image is saved to disk.
    args: 
      y_true:    true label of the data, with shape (nsamples,)
      y_pred:    prediction of the data, with shape (nsamples,)
      filename:  filename of figure file to save
      labels:    string array, name the order of class labels in the confusion matrix.
                 use `clf.classes_` if using scikit-learn models.
                 with shape (nclass,).
      ymap:      dict: any -> string, length == nclass.
                 if not None, map the labels & ys to more understandable strings.
                 Caution: original y_true, y_pred and labels must align.
      figsize:   the size of the figure plotted.
    """
    if ymap is not None:
        y_pred = [ymap[yi] for yi in y_pred]
        y_true = [ymap[yi] for yi in y_true]
        labels = [ymap[yi] for yi in labels]
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.1f%%\n%d/%d' % (p, c, s)
            elif c == 0:
                annot[i, j] = ''
            else:
                annot[i, j] = '%.1f%%\n%d' % (p, c)
    cm = pd.DataFrame(cm, index=labels, columns=labels)
    cm.index.name = 'Actual'
    cm.columns.name = 'Predicted'
    fig, ax = plt.subplots()
    sns.heatmap(cm, annot=annot, fmt='', ax=ax)
    #plt.savefig(filename)
    plt.show()

cm_analysis(y_test, y_val_pred_bl, labels, ymap=None, figsize=(5,5))




In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt     
from sklearn.metrics import confusion_matrix


labels = [0,1]
cm = confusion_matrix(y_test, y_val_pred_bl, labels)

ax= plt.subplot()
sns.heatmap(cm, annot=True, fmt="g", ax = ax,); #annot=True to annotate cells

# labels, title and ticks
ax.set_xlabel('Predicted labels');
ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(labels); 
ax.yaxis.set_ticklabels(labels);

## Predicting Joker

In [ ]:
'''
print(dfTweetsRelevantTopic.head(3))
testJokerSamples = dfTweetsRelevantTopic["Text"]


testJokerSamples = testJokerSamples.dropna()
#print(testSamples[1])
tokenizerObj.fit_on_texts(testJokerSamples)
testJokerSamplesTokens = tokenizerObj.texts_to_sequences( testJokerSamples )
testJokerSamplesTokensPad = pad_sequences( testJokerSamplesTokens, maxlen = maxLen)
print(len(testJokerSamplesTokensPad))

Xnew = testJokerSamplesTokensPad
ynew = modelBiLSTM.predict_classes(Xnew)
print(type(ynew))
print(ynew[0])


numPositiveBilstm = 0
numNegativeBilstm = 0
for i in range(0, len(ynew)):
  if ynew[i]==0:
    numNegativeBilstm = numNegativeBilstm + 1
  elif  ynew[i] == 1:
    numPositiveBilstm = numPositiveBilstm + 1
print('Positive', numPositiveBilstm)
print('Negatives', numNegativeBilstm)
'''

# CNN

In [ ]:


# set parameters:
max_features = numWords
#maxlen = 400


print('Build model...')
modelConv1 = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
modelConv1.add(Embedding(numWords, embeddingDim , 
                      embeddings_initializer = Constant( embeddingMatrix), 
                      input_length=maxLen,
                      trainable = isTrainable))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:

modelConv1.add(Conv1D(filters=filtersNum,
                 kernel_size=kernelSize,
                 padding='same',
                 activation='relu',
                 strides=1))
modelConv1.add(Dropout(0.5))

modelConv1.add(MaxPooling1D(pool_size=poolSize))
modelConv1.add(Dropout(0.5))

modelConv1.add(Flatten())
modelConv1.add(Dense(10, activation='relu'))
modelConv1.add(Dropout(0.5))

modelConv1.add(BatchNormalization())

modelConv1.add(Dense(1, activation='sigmoid'))








opt = SGD(lr=0.1)


modelConv1.compile(loss='binary_crossentropy',
              optimizer= 'adam',
              metrics=['accuracy'])

#modelConv1.summary()


historyConv1=modelConv1.fit(x_train_pad, y_train,
          batch_size=batchSize,
          epochs=epochsNum,
          validation_data=(x_test_pad, y_test), verbose = 1)


In [ ]:
testLossConv, testAccConv = modelConv1.evaluate(x_test_pad, y_test, batch_size=batchSize)
print('Test loss:', testLossConv)
print('Test accuracy:', testAccConv)

In [ ]:
trainLossConv, trainAccConv = modelConv1.evaluate(x_train_pad, y_train, batch_size=batchSize)
print('Train loss:', trainLossConv)
print('Train accuracy:', trainAccConv)

In [ ]:
#modelConv1.save(conv1ModelTxt)
#modelConv1.save(conv1mModelH)

In [ ]:
modelConv1.summary()

In [ ]:
plt.title('Accuracy')
plt.plot(historyConv1.history['accuracy'], label='train')
plt.plot(historyConv1.history['val_accuracy'], label='test')
plt.legend()
plt.show();

In [ ]:
plt.title('Loss')
plt.plot(historyConv1.history['loss'], label='train')
plt.plot(historyConv1.history['val_loss'], label='test')
plt.legend()
plt.show();

In [ ]:
y_val_pred_c =modelConv1.predict_classes(x_test_pad)
precisionConv1, recallConv1, fscoreConv1, supportConv1 = score(y_test, y_val_pred_c)

print('precision: {}'.format(precisionConv1))
print('recall: {}'.format(recallConv1))
print('fscore: {}'.format(fscoreConv1))
print('support: {}'.format(supportConv1))

In [ ]:
from sklearn.metrics import cohen_kappa_score

cohen_score = cohen_kappa_score(y_test, y_val_pred_c)

print(cohen_score)

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
labels = [0,1]
def cm_analysis(y_true, y_pred, labels, ymap=None, figsize=(10,10)):
    """
    Generate matrix plot of confusion matrix with pretty annotations.
    The plot image is saved to disk.
    args: 
      y_true:    true label of the data, with shape (nsamples,)
      y_pred:    prediction of the data, with shape (nsamples,)
      filename:  filename of figure file to save
      labels:    string array, name the order of class labels in the confusion matrix.
                 use `clf.classes_` if using scikit-learn models.
                 with shape (nclass,).
      ymap:      dict: any -> string, length == nclass.
                 if not None, map the labels & ys to more understandable strings.
                 Caution: original y_true, y_pred and labels must align.
      figsize:   the size of the figure plotted.
    """
    if ymap is not None:
        y_pred = [ymap[yi] for yi in y_pred]
        y_true = [ymap[yi] for yi in y_true]
        labels = [ymap[yi] for yi in labels]
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.1f%%\n%d/%d' % (p, c, s)
            elif c == 0:
                annot[i, j] = ''
            else:
                annot[i, j] = '%.1f%%\n%d' % (p, c)
    cm = pd.DataFrame(cm, index=labels, columns=labels)
    cm.index.name = 'Actual'
    cm.columns.name = 'Predicted'
    fig, ax = plt.subplots()
    sns.heatmap(cm, annot=annot, fmt='', ax=ax)
    #plt.savefig(filename)
    plt.show()

cm_analysis(y_test, y_val_pred_c, labels, ymap=None, figsize=(5,5))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt     
from sklearn.metrics import confusion_matrix


labels = [0,1]
cm = confusion_matrix(y_test, y_val_pred_c, labels)

ax= plt.subplot()
sns.heatmap(cm, annot=True, fmt="g", ax = ax,); #annot=True to annotate cells

# labels, title and ticks
ax.set_xlabel('Predicted labels');
ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(labels); 
ax.yaxis.set_ticklabels(labels);

# Evaluation

## Accuracy

In [ ]:
dicTestAccuracy={
      "LSTM"      :("{:.4f}".format(testAccLSTM)),
      "Bilstm":("{:.4f}".format(testAccBilstm)),
      "GRU"      :("{:.4f}".format(testAccGRU)),
      "CNN"     :("{:.4f}".format(testAccConv)),
      "CNN+LSTM"      :("{:.4f}".format(testAccCL)),
      "CNN+GRU"      :("{:.4f}".format(testAccCGRU)),
      "CNN+BiLSTM"      :("{:.4f}".format(testAccCBiLSTM))
      }

In [ ]:
dfTestAccuracy = pd.DataFrame.from_dict(dicTestAccuracy, orient='index', columns=["Accuracy"] )
dfTestAccuracy

In [ ]:
dfTestAccuracy["Accuracy"] = dfTestAccuracy["Accuracy"].astype(float)
dfTestAccuracy["Accuracy"].plot(kind="bar", grid=True , color=tuple(["g", "b", "r", "y", "k"]), width=0.3)
plt.title("Accuracy")

## Precision

In [ ]:
precisionLSTM

In [ ]:
dicPrecision={
      "LSTM"      :("{:.4f}".format(precisionLSTM[0])),
      "Bilstm":("{:.4f}".format(precisionBilstm[0])),
      "GRU"      :("{:.4f}".format(precisionGRU[0])),
      "CNN"     :("{:.4f}".format(precisionConv1[0])),
      "CNN+LSTM"      :("{:.4f}".format(precisionCL[0])),
      "CNN+GRU"      :("{:.4f}".format(precisionCGRU[0])),
      "CNN+BiLSTM"      :("{:.4f}".format(precisionCBiLSTM[0]))
      }


In [ ]:
dfPrecision = pd.DataFrame.from_dict(dicPrecision, orient='index', columns=["Precision"] )
dfPrecision

In [ ]:
dfPrecision["Precision"] = dfPrecision["Precision"].astype(float)
dfPrecision["Precision"].plot(kind="bar", grid=True , color=tuple(["g", "b", "r", "y", "k"]), width=0.3)
plt.title("Precision")

## Recall

In [ ]:
dicRecall={
      "LSTM"      :("{:.4f}".format(recallLSTM[0])),
      "Bilstm":("{:.4f}".format(recallBilstm[0])),
      "GRU"      :("{:.4f}".format(recallGRU[0])),
      "CNN"     :("{:.4f}".format(recallConv1[0])),
      "CNN+LSTM"      :("{:.4f}".format(recallCL[0])),
      "CNN+GRU"      :("{:.4f}".format(recallCGRU[0])),
      "CNN+BiLSTM"      :("{:.4f}".format(recallCBiLSTM[0]))
      }

In [ ]:
dfRecall = pd.DataFrame.from_dict(dicRecall, orient='index', columns=["Recall"] )
dfRecall

In [ ]:
dfRecall["Recall"] = dfRecall["Recall"].astype(float)
dfRecall["Recall"].plot(kind="bar", grid=True , color=tuple(["g", "b", "r", "y", "k"]), width=0.3)
plt.title("Recall")

## Fscore

In [ ]:
dicFscore={
      "LSTM"      :("{:.4f}".format(fscoreLSTM[0])),
      "Bilstm":("{:.4f}".format(fscoreBilstm[0])),
      "GRU"      :("{:.4f}".format(fscoreGRU[0])),
      "CNN"     :("{:.4f}".format(fscoreConv1[0])),
      "CNN+LSTM"      :("{:.4f}".format(fscoreCL[0])),
      "CNN+GRU"      :("{:.4f}".format(fscoreCGRU[0])),
      "CNN+BiLSTM"      :("{:.4f}".format(fscoreCBiLSTM[0]))
      }

In [ ]:
dfFscore = pd.DataFrame.from_dict(dicFscore, orient='index', columns=["Fscore"] )
dfFscore

In [ ]:
dfFscore["Fscore"] = dfFscore["Fscore"].astype(float)
dfFscore["Fscore"].plot(kind="bar", grid=True , color=tuple(["g", "b", "r", "y", "k"]), width=0.3)
plt.title("Fscore")

##ALL

In [ ]:
dicALL={
      "Accuracy":[
                  ("{:.4f}".format(testAccLSTM)),
                  ("{:.4f}".format(testAccBilstm)),
                  ("{:.4f}".format(testAccGRU)),
                  ("{:.4f}".format(testAccConv)),
                  ("{:.4f}".format(testAccCL)),
                  ("{:.4f}".format(testAccCGRU)),
                  ("{:.4f}".format(testAccCBiLSTM))
                 ],
      
      "Precision":[
                  ("{:.4f}".format(precisionLSTM[0])),
                  ("{:.4f}".format(precisionBilstm[0])),
                  ("{:.4f}".format(precisionGRU[0])),
                  ("{:.4f}".format(precisionConv1[0])),
                  ("{:.4f}".format(precisionCL[0])),
                  ("{:.4f}".format(precisionCGRU[0])),
                  ("{:.4f}".format(precisionCBiLSTM[0])) 
                 ],
      
      "Recall":[
                ("{:.4f}".format(recallLSTM[0])),
                ("{:.4f}".format(recallBilstm[0])),
                ("{:.4f}".format(recallGRU[0])),
                ("{:.4f}".format(recallConv1[0])),
                ("{:.4f}".format(recallCL[0])),
                ("{:.4f}".format(recallCGRU[0])),
                ("{:.4f}".format(recallCBiLSTM[0]))
                 ],
        
      "F1":[
            ("{:.4f}".format(fscoreLSTM[0])),
            ("{:.4f}".format(fscoreBilstm[0])),
            ("{:.4f}".format(fscoreGRU[0])),
            ("{:.4f}".format(fscoreConv1[0])),
            ("{:.4f}".format(fscoreCL[0])),
            ("{:.4f}".format(fscoreCGRU[0])),
            ("{:.4f}".format(fscoreCBiLSTM[0]))
           ],
     "Loss":[
            ("{:.4f}".format(testLossLSTM)),
            ("{:.4f}".format(testLossBilstm)),
            ("{:.4f}".format(testLossGRU)),
            ("{:.4f}".format(testLossConv)),
            ("{:.4f}".format(testLossCL)),
            ("{:.4f}".format(testLossCGRU)),
            ("{:.4f}".format(testLossCBiLSTM))
           ]
        
        
        }


dfAllMeasures=pd.DataFrame.from_dict( dicALL , orient='index')#,index=['SIMPLE RNN','LSTM MODEL','GRU MODEL','SVR MODEL','SARIMAX MODEL'],
                                 #columns=["MSE","MAE"] )
dfAllMeasures.columns=['LSTM','Bilstm','GRU','Conv1','CNN+LSTM', 'CNN+GRU', 'CNN+BiLSTM']
dfAllMeasures = dfAllMeasures.T
dfAllMeasures